In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from scipy import spatial

from sklearn.metrics import mean_squared_error, auc, mean_absolute_error 
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.metrics import mean_squared_error, roc_auc_score
import time
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LogisticRegression,SGDRegressor, LinearRegression,Ridge
from sklearn.metrics.pairwise import cosine_similarity

%matplotlib inline
import seaborn as sns
sns.set_theme()

In [ ]:
train_dir = "/kaggle/input/commonlitreadabilityprize"
train_path = os.path.join(train_dir, "train.csv")
test_path = os.path.join(train_dir, "test.csv")

glove_files = ["glove.6B.300d.txt","glove.6B.200d.txt", "glove.6B.100d.txt", "glove.6B.50d.txt"]
glove_path_dir = "../input/glove6b/"
sample = os.path.join(train_dir, "sample_submission.csv")
df = pd.read_csv(train_path)
df.head()

df_test = pd.read_csv(test_path)

df_sample = pd.read_csv(sample)

In [ ]:
# array  =[]; glove_embedding = {}; count=0

# sentence = "example sentence for word emnedding"

# with open(glove_path_dir+glove_files[-1], "r") as f:
#     for line in f:
#         k = line.split()
#         if k[0].isalpha():
#             glove_embedding[k[0]] = np.asarray(k[1:], np.float32)
            

In [ ]:
# # 
# def find_closest_embeddings(embedding, embeddings_dict= glove_embedding):
#     return sorted(embeddings_dict.keys(), key=lambda word: spatial.distance.euclidean(embeddings_dict[word], embedding))[:10]

# embed_list = find_closest_embeddings(glove_embedding['king'])
# embed_list

In [ ]:
def print_results_original(results,original ):
    #  Convert the results to a string, and word-wrap them.
    results = re.sub(r"(.{,70})\s", r"\1\n", results + " ").rstrip()

    # Convert the original to a string, and word wrap it.
    original = re.sub(r"(.{,70})\s", r"\1\n", original + " ").rstrip()

    # Print the results.
    print("Original " ,len(original))
    print("Reults ",len(results))
    print("-Original-".center(70).replace(" ", "*").replace("-", " "))
    print(original)
    print("-Results-".center(70).replace(" ", "*").replace("-", " "))
    print(results)
    print("*" * 70)

In [ ]:
# from nltk import wordnet 
# import regex as re
# from nltk import stem

# stemmer = stem.PorterStemmer()
# treebank = df.iloc[:,3]
# stemmed = []

# for i in range(df.shape[0]):
#     for word in df.iloc[i,3]:
#         stemmed.append(stemmer.stem(word))
#     results = "".join(stemmed)
#     if len(df.iloc[:,3])>len(results):
#         if i<1:
#             print_results_original(results, df.iloc[i,3])
#         df.iloc[i,3]= results
#     stemmed = []
# #

In [ ]:
df.iloc[:,3] = df.iloc[:,3].str.replace("[:;.,<>/-_~`!@#$%^&*?[]{}|-+0-9]", "", regex = True)
df.iloc[16, 3]

In [ ]:
def plot_graph(data):
    tar_s = np.unique(data['target']).shape

    fig,ax = plt.subplots(1,2,figsize=(15,8))
    sns.scatterplot(x = np.arange(tar_s[0]), y =data['target'],ax=ax[0] )
    sns.scatterplot(x = np.arange(tar_s[0]), y =data['standard_error'],ax=ax[1] )
    return fig, ax
# plot_graph(df)
    # A outlier could easily be drop with IQR and pandas dataframe filter method
    # There is one outlier in standard error at index = 0

In [ ]:
def get_validate_data(X, y):
    X_train, X_test, y_train,y_test = train_test_split(X, y, test_size= 0.2)
    print(X_train.shape, X_test.shape, y_train.shape,y_test.shape)
    return X_train, X_test, y_train,y_test


In [ ]:
df_describe = df.describe()
Q3 = df_describe.loc['75%']; Q1 = df_describe.loc['25%']
IQR = Q3 - Q1

upper_iqr = Q3 +1.5*IQR
lower_iqr  = Q1 - 1.5*IQR

# print(f"Condition for outliers is {df['target']<=  upper_iqr and df['target']<= lower_iqr}")
df['target']
m = map(lambda x: x>= lower_iqr, df['target'])


df_upper = df.groupby("target").filter(lambda x: (x['standard_error']> upper_iqr['standard_error'] ))
df_lower = df.groupby("target").filter(lambda x: (x['standard_error']< lower_iqr['standard_error'] ))

df_upper2 = df.groupby("standard_error").filter(lambda x: (x['target']> upper_iqr['target'] ))
df_lower2 = df.groupby("standard_error").filter(lambda x: (x['target']< lower_iqr['target'] ))

df.drop(index= df_upper.index, axis=0, inplace=True)
df.drop(index = df_lower.index, axis=0, inplace=True)
df.drop(index = df_upper2.index, axis=0, inplace=True)
df.drop(index = df_lower2.index, axis=0, inplace=True)
# ploting target data after removing outlier

In [ ]:
# plot_graph(df)

df.head(5)

df['target_std'] = np.divide(df.iloc[:, 4], df.iloc[:, 5])
df['target_std_m'] = np.subtract(df.iloc[:, 4], df.iloc[:, 5])

mean_err = np.mean(df.iloc[:, 5])
df['target_std_mean'] = np.divide(df.iloc[:, 4], mean_err)
df.head(5)

In [ ]:
def rmse_score(model, X, y):
    y_pred = model.predict(X)
    score = cross_val_score(model, X, y, n_jobs= -1, cv= 5); m = mean_squared_error(y, y_pred)
    print(f"Cross_val_Score for X_train prediction:\t{score}\nmean squared error:\t{m}")
    return score, m 

In [ ]:
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
d = vectorizer.fit_transform(df['excerpt'])

y_train = df.iloc[:, 4]
y_train1 = df.iloc[:, 6]
y_train2 = df.iloc[:, 7]
y_train3 = df.iloc[:, 8]

X_train = d.toarray()

X_train_, X_test_, y_train,y_test = get_validate_data(X_train, y_train)
X_train1, X_test1, y_train1,y_test1 = get_validate_data(X_train, y_train1)
X_train2, X_test2, y_train2,y_test2 = get_validate_data(X_train, y_train2)
X_train3, X_test3, y_train3,y_test3 = get_validate_data(X_train, y_train3)

X_train.shape, y_train.shape

In [ ]:
# word_dict = {}
# words = vectorizer.get_feature_names()
# skip_word = 0; X_arr = np.array([])
# for i in words:
#     try:
# #         X_arr = np.append(X_arr, glove_embedding[i])
#         word_dict[i] = (glove_embedding[i])
#     except KeyError:
#         skip_word +=1
# print("Skip word\t%s"%skip_word)
# print("len of word dict\t%s"%len(word_dict))
# print("X array shape\t%s"%X_arr.shape)

In [ ]:
# word_array = np.array(list(word_dict.values()))

In [ ]:
'''
It is slow  model with accuracy of (rmse ) is 0.75, which is not good to enough to get good rank under 200
'''
from sklearn.gaussian_process import GaussianProcessRegressor

model = GaussianProcessRegressor(n_restarts_optimizer= 10 )

model.fit(X_train_, y_train)


test_score, test_mean = rmse_score(model, X_test_,y_test)
train_score, train_mean = rmse_score(model, X_train_,y_train)


In [ ]:
# it has low accuracy than gauss, 0.756

# ridge_estimator = Ridge(alpha= 0.5)

# ridge_estimator.fit(X_train_, y_train)

# test_score_, test_mean_ = rmse_score(ridge_estimator, X_test_,y_test)
# train_score_, train_mean_ = rmse_score(ridge_estimator, X_train_,y_train)


In [ ]:
# print("Guassian estimator rmse score",rmse_score(model, X_test_, y_test))

# print("Linear Model estimator score", rmse_score(ridge_estimator, X_train_, y_train))
# print("Linear Model estimator score", rmse_score(ridge_estimator, X_test_, y_test))

In [ ]:
# fig, ax = plt.subplots(1,2, figsize= (15, 8))
# ax[0].plot(test_score_, 'y')
# ax[0].plot(train_score_, 'r')
# ax[0].set_title("cross_val_score")

# ax[0].annotate("Validation data",xy= (2.0,0.410), arrowprops=dict(facecolor='red', shrink=5),fontsize=16,horizontalalignment='left')
# ax[0].annotate("training data",xy= (2.0,0.30), arrowprops=dict(facecolor='red', shrink=5),fontsize=16,horizontalalignment='left')

# ax[1].plot(test_score, 'y')
# ax[1].plot(train_score, 'r')
# ax[1].set_title("cross_val_score")

# ax[1].annotate("Validation data",xy= (2.0,0.410), arrowprops=dict(facecolor='red', shrink=5),fontsize=16,horizontalalignment='left')
# ax[1].annotate("training data",xy= (2.0,0.30), arrowprops=dict(facecolor='red', shrink=5),fontsize=16,horizontalalignment='left')

# # from graph, it is clear, our model is overfitting data very high, could be fix with l1 or l2 

In [ ]:
def predict_from_gauss(model, data, num):
    p = data.iloc[num]['excerpt']
    p = vectorizer.transform([p])
    pred = model.predict(p.toarray())
    print(pred)
    return pred[0]

def predict_traget_(model, data, save_file=False):
    data['target']=0
    for i in range(data.shape[0]):
        data.iloc[i, -1] = predict_from_gauss(model, data, i)
    for i in ['url_legal','license','excerpt']:
        try:
            df_test.drop(i, inplace=True, axis=1)
        except KeyError:
            pass
    if save_file:
        data.to_csv("submission.csv", index=False)
    return f"Successfully test data submitted or written in output directory\tsubmission.csv"
    

predict_traget_(model, df_test, save_file=True )

In [ ]:
pd.read_csv("submission.csv")